In [ ]:
## libraries
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
import nltk
from nltk.corpus import wordnet
import copy

In [ ]:
## data loading
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])

In [ ]:
#ftd.dtype.names

In [ ]:
#changing or adding names for the dtypes
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')
ftd.dtype.names = ('img_id', 'description')
len(ftd['img_id'])
len(set(ftd['img_id']))

#fe.dtype.names
#fe[971]

In [ ]:
## first convert strings to list of rows equal to rows in fe
def mk_list(str_vector):
    des_byid = str_vector
    all_x = []
    for i in des_byid:
        j = i.split()
        #if len(j) > 0:
        all_x.append(j)

    return all_x


In [ ]:
all_x = mk_list(fe['pre_que'])
len(all_x)


In [ ]:
## Now for each row in the list above, find description and add it
all_i_txt = []

for rown in range(len(all_x)):
    #if rown not in bl:
        #print(all_x.index)
    int_i_txt = []
    for img in all_x[rown]:
        if img in ftd['img_id']:
            d = str(ftd['description'][list(ftd['img_id']).index(img)])
            int_i_txt.append(str(d))
    all_i_txt.append(int_i_txt)
        
            
            

In [ ]:
## For each question, find the synonyms for all the adjectives, NN, NNS and CD. Then add that list of synonyms in the 
##column called question 

new_table_fd = [] ## for frequency distribution
new_que_syn = []
for rown in qa['que'] :
    #print(rown[-2])
    #new_row = []
    sents = nltk.sent_tokenize(rown.strip())
    
    lw = [nltk.word_tokenize(s) for s in sents]
    #p =[nltk.pos_tag(w) for w in lw]
    x = [kv[0] for w in lw for kv in nltk.pos_tag(w) if kv[1] in ['JJ', 'NN', 'NNS', 'CD']]
    fd = [kv[1] for w in lw for kv in nltk.pos_tag(w)]
    #y = [wordnet.synsets(words) for words in x]
    syn = [j.name() for words in x for i in wordnet.synsets(words) for j in i.lemmas()]
    new_que_syn.append(list(set(syn)))
    new_table_fd.extend([kv[0] for w in lw for kv in nltk.pos_tag(w)])

In [ ]:
## add question and answer to each data point also convert the list of synonyms in the question to a string
for it in range(len(all_i_txt)):
    all_i_txt[it].extend([' '.join(map(str, new_que_syn[it])), qa['ans'][it].strip()])
    
    

In [ ]:
## check addition of above objects
for i in range(len(all_i_txt[0:2])):
    print(all_i_txt[i][-4:])

In [ ]:
#for i in set(bl):
#    print(all_i_txt[i][-2:])

In [ ]:
#for rown,qs in zip(all_i_txt[0:2],new_que_syn[0:2]):
#    rown[-2] = qs
#    print(rown[-2])

In [ ]:
#fd1 = nltk.FreqDist(new_table_fd[0:20])
fd1 = nltk.FreqDist(new_table_fd)
fd1.most_common()

In [ ]:
#print(all_i_txt[0])
##basic model
tf = TfidfVectorizer()

In [ ]:
all_sim_tf = []
for l in range(len(all_i_txt)):
    H = tf.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim_tf.append(sim)

In [ ]:
#similarities[-2] meaning taking only the array for question
all_sim_q_tf = []
for i in range(len(all_sim_tf)):
    all_sim_q_tf.append(all_sim_tf[i][-2][:-2])
print(len(all_sim_q_tf))
#print(all_sim_q[971])
#print(len(all_sim))

In [ ]:
all_cheat = mk_list(fe['curr_elim'])

In [ ]:
cheat_code = zip(all_sim_q_tf, all_cheat)

In [ ]:
## Sort all the above similarities with argsort which gives indices
## or we should not take top5 and make a threshold above which we will include it in the list of output?
sorted_ind = []
for si, ln in cheat_code:
    y_top = si.argsort()[::-1][:len(ln)]
    sorted_ind.append(y_top)
#sorted_ind[971]


In [ ]:
top_img = zip(all_x, sorted_ind)


In [ ]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred
    
    

In [ ]:
pred_img = mk_imgid(top_img)

In [ ]:
#print(all_cheat[0:3])
#v = mk_imgid(top_img)
#print(w)

In [ ]:
acc_in = zip(all_cheat, pred_img)

In [ ]:
acc_all = []
for a,p in acc_in:
    #print(a,p)
    if len(a) == 0:
        acc_i = 0
    else:
        acc_i = len(set(a).intersection(set(p)))/len(set(a))
    acc_all.append(acc_i)

In [ ]:
#len(pred_img)
print(acc_all)

In [ ]:
np.mean(np.array(acc_all))

In [ ]:
cv = CountVectorizer()

In [ ]:
## all similarities
all_sim_cv = []
for l in range(len(all_i_txt)):
    H = cv.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim_cv.append(sim)

In [ ]:
#similarities[-2] meaning taking only the array for question
all_sim_q_cv = []
for i in range(len(all_sim_cv)):
    all_sim_q_cv.append(all_sim_cv[i][-2][:-2])
print(len(all_sim_q_cv))
#print(all_sim_q[971])
#print(len(all_sim))

In [ ]:
all_cheat = mk_list(fe['curr_elim'])

In [ ]:
cheat_code = zip(all_sim_q_cv, all_cheat)## Sort all the above similarities with argsort which gives indices
## or we should not take top5 and make a threshold above which we will include it in the list of output?
sorted_ind = []
for si, ln in cheat_code:
    y_top = si.argsort()[::-1][:len(ln)]
    sorted_ind.append(y_top)
#sorted_ind[971]


In [ ]:
top_img = zip(all_x, sorted_ind)

In [ ]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred
    
    

In [ ]:
pred_img = mk_imgid(top_img)

In [ ]:
acc_in = zip(all_cheat, pred_img)

In [ ]:
acc_all = []
for a,p in acc_in:
    #print(a,p)
    if len(a) == 0:
        acc_i = 0
    else:
        acc_i = len(set(a).intersection(set(p)))/len(set(a))
    acc_all.append(acc_i)

In [ ]:
print(acc_all)

In [ ]:
np.mean(np.array(acc_all))